# Extração do Mercado Livre

## 1. Obtenção dos links dos produtos
Primeiramente, vamos extrair os links de todos os produtos.

### Instalação de bibliotecas

In [11]:
%pip install beautifulsoup4
%pip install requests
%pip install pandas
%pip install numpy


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### Importando bibliotecas

In [12]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time

### Lista de categorias
Cada categoria tem um nome e um link, o qual será usado para obter links dos produtos em cada página.

In [13]:
category_list = [
    {
        'name': 'Celulares',
        'link': 'https://www.mercadolivre.com.br/ofertas?container_id=MLB779535-1&domain_id=MLB-CELLPHONES&page='
    }, 
    {
        'name': 'Notebooks',
        'link': 'https://www.mercadolivre.com.br/ofertas?container_id=MLB779536-1&domain_id=MLB-NOTEBOOKS&page='
    },
    {
        'name': 'Tênis',
        'link': 'https://www.mercadolivre.com.br/ofertas?container_id=MLB779537-1&domain_id=MLB-SNEAKERS&page='
    },
    {
        'name': 'Fones',
        'link': 'https://www.mercadolivre.com.br/ofertas?container_id=MLB779538-1&domain_id=MLB-HEADPHONES&page='
    },
    {
        'name': 'TVs',
        'link': 'https://www.mercadolivre.com.br/ofertas?container_id=MLB779539-1&domain_id=MLB-TELEVISIONS&page='
    },
    {
        'name': 'Ferramentas',
        'link': 'https://www.mercadolivre.com.br/ofertas?container_id=MLB779540-1&domain_id=MLB-WELDING_MACHINES&page='
    },
    {
        'name': 'Smartwatches',
        'link': 'https://www.mercadolivre.com.br/ofertas?container_id=MLB779541-1&domain_id=MLB-SMARTWATCHES&page='
    },
    {
        'name': 'Caixas de som',
        'link': 'https://www.mercadolivre.com.br/ofertas?container_id=MLB779542-1&domain_id=MLB-SPEAKERS&page='
    },
    {
        'name': 'Perfumes',
        'link': 'https://www.mercadolivre.com.br/ofertas?container_id=MLB779543-1&domain_id=MLB-PERFUMES&page='
    },
    {
        'name': 'Bebês',
        'link': 'https://www.mercadolivre.com.br/ofertas?container_id=MLB773331-2&deal_ids=MLB773331-2&page='
    }
]

### Funções para requisição de páginas e extração de links

In [23]:
def request_page(url):
    success = False
    page = None
    while not success:
        try:
            response = requests.get(url, allow_redirects=False)
            if response.is_redirect:
                # Ignorando este url por ter redirecionado
                break
            page = BeautifulSoup(response.text)
            success = True
        except requests.exceptions.RequestException as e:
            print(f"Erro na requisição de {url}")
            time.sleep(60)
    return page

def get_links_from_page(page):
    links = []
    if page != None: 
        link_tags = page.find_all('a',  class_ = 'promotion-item__link-container')
        for tag in link_tags:
            links.append(tag['href'])
    return links


    

### Obtendo links dos produtos de todas as categorias

In [24]:
product_links = []
# Para cada categoria
for category in category_list:
    # Gere uma lista de páginas de 1 até 20
    page_list = [f"{category['link']}{i+1}" for i in range(20)]
    category_product_total = 0
    # Para cada página desta categoria
    for page in page_list:
        page_doc = request_page(page)
        links = get_links_from_page(page_doc)
        # Já passou da última página? Pare esta categoria
        if (len(links) == 0):
            break
        # Adicione os links da página na lista de links dos produtos
        product_links.extend(links)
        category_product_total += len(links)
    print(f"Produtos da categoria {category['name']} adicionados com sucesso (Total = {category_product_total})")

Produtos da categoria Celulares adicionados com sucesso (Total = 960)
Produtos da categoria Notebooks adicionados com sucesso (Total = 738)
Produtos da categoria Tênis adicionados com sucesso (Total = 960)
Produtos da categoria Fones adicionados com sucesso (Total = 960)
Produtos da categoria TVs adicionados com sucesso (Total = 267)
Produtos da categoria Ferramentas adicionados com sucesso (Total = 720)
Produtos da categoria Smartwatches adicionados com sucesso (Total = 864)
Produtos da categoria Caixas de som adicionados com sucesso (Total = 960)
Produtos da categoria Perfumes adicionados com sucesso (Total = 960)
Produtos da categoria Bebês adicionados com sucesso (Total = 960)


In [26]:
print(f"Total de produtos: {len(product_links)}")

Total de produtos: 8349


### Salvando links dos produtos em um arquivo

In [25]:
arr = np.asarray(product_links)
np.savetxt('links.csv', arr, fmt = '%s', delimiter="\n")  